In [ ]:
# 중심축 기반 부피 계산
import trimesh
import numpy as np
from scipy.spatial.transform import Rotation
from scipy.optimize import least_squares

# GLB 로드 및 회전
scene = trimesh.load('/data/ephemeral/home/project/output/scene.glb')
arcore_y = np.array([0, 1, 0])
gl_to_cv = np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]])
A = np.array(scene.metadata.get('hf_alignment', np.eye(4))).reshape(4, 4)
gravity = (A[:3, :3] @ gl_to_cv @ arcore_y)
gravity = gravity / np.linalg.norm(gravity)

R = Rotation.align_vectors([[0, 0, 1]], [gravity])[0]
T = np.eye(4)
T[:3, :3] = R.as_matrix()
for geom in scene.geometry.values():
    geom.apply_transform(T)

# 포인트 클라우드 가져오기
points = None
for name, geometry in scene.geometry.items():
    if isinstance(geometry, trimesh.PointCloud):
        points = geometry.vertices
        break

# 원점 근처 필터링 (중심축 계산용 - 거리만 제한, 높이 제한 없음)
distances = np.linalg.norm(points[:, :2], axis=1)
heights = points[:, 2]
mask = distances < 0.2  # 높이 제한 제거
filtered_points = points[mask]
filtered_heights = filtered_points[:, 2]

# 높이로 정렬
sorted_idx = np.argsort(filtered_heights)
sorted_points = filtered_points[sorted_idx]
sorted_heights = filtered_heights[sorted_idx]

def refine_circle_with_least_squares(inlier_points):
    if len(inlier_points) < 3:
        return None
    
    center_init = inlier_points.mean(axis=0)
    radius_init = np.mean(np.linalg.norm(inlier_points - center_init, axis=1))
    
    def residuals(params):
        cx, cy, r = params
        distances = np.linalg.norm(inlier_points - [cx, cy], axis=1)
        return distances - r
    
    try:
        result = least_squares(
            residuals, 
            [center_init[0], center_init[1], radius_init],
            bounds=([-0.5, -0.5, 0.001], [0.5, 0.5, 0.2])
        )
        cx, cy, r = result.x
        if 0.01 < r < 0.15:
            return {'center': np.array([cx, cy]), 'radius': r}
    except:
        pass
    return None

def fit_circle_ransac(points_2d, n_iterations=50, threshold=0.002, min_inliers=10):
    if len(points_2d) < 3:
        return None
    
    best_circle = None
    best_score = 0
    
    for _ in range(n_iterations):
        sample_indices = np.random.choice(len(points_2d), 3, replace=False)
        p1, p2, p3 = points_2d[sample_indices]
        
        try:
            A = np.array([
                [2*(p2[0]-p1[0]), 2*(p2[1]-p1[1])],
                [2*(p3[0]-p1[0]), 2*(p3[1]-p1[1])]
            ])
            b = np.array([
                p2[0]**2 - p1[0]**2 + p2[1]**2 - p1[1]**2,
                p3[0]**2 - p1[0]**2 + p3[1]**2 - p1[1]**2
            ])
            
            center = np.linalg.solve(A, b)
            radius = np.linalg.norm(p1 - center)
            
            if not (0.01 < radius < 0.15):
                continue
            
            distances = np.abs(np.linalg.norm(points_2d - center, axis=1) - radius)
            inliers = distances < threshold
            n_inliers = np.sum(inliers)
            
            if n_inliers < min_inliers:
                continue
            
            score = n_inliers / len(points_2d)
            
            if score > best_score:
                best_score = score
                inlier_points = points_2d[inliers]
                refined_circle = refine_circle_with_least_squares(inlier_points)
                
                if refined_circle is not None:
                    best_circle = {
                        'center': refined_circle['center'],
                        'radius': refined_circle['radius'],
                        'score': score,
                        'n_inliers': n_inliers
                    }
        except np.linalg.LinAlgError:
            continue
    
    return best_circle

def find_cup_circle_ransac(slice_2d, origin=np.array([0, 0]), 
                           n_iterations=50, threshold=0.005):
    if len(slice_2d) < 10:
        return None
    
    circle = fit_circle_ransac(slice_2d, n_iterations=n_iterations, 
                               threshold=threshold, min_inliers=10)
    
    if circle is None:
        return None
    
    dist_from_origin = np.linalg.norm(circle['center'] - origin)
    final_score = circle['score'] - dist_from_origin * 1.5
    
    circle['final_score'] = final_score
    circle['dist_from_origin'] = dist_from_origin
    
    if final_score > 0.2 and dist_from_origin < 0.15:
        return circle
    
    return None

# 1단계: 중심축 계산을 위한 원 찾기
z_range = np.linspace(sorted_heights.min(), sorted_heights.max(), 20)
thickness = 0.0003
circle_data = []

for z in z_range:
    h_min = z - thickness
    h_max = z + thickness
    start_idx = np.searchsorted(sorted_heights, h_min)
    end_idx = np.searchsorted(sorted_heights, h_max)
    
    if end_idx - start_idx < 10:
        continue
    
    slice_points = sorted_points[start_idx:end_idx]
    slice_2d = slice_points[:, :2]
    
    cup_circle = find_cup_circle_ransac(slice_2d, n_iterations=30, threshold=0.005)
    
    if cup_circle is not None:
        circle_data.append({
            'z': z,
            'center': cup_circle['center'],
            'radius': cup_circle['radius'],
            'final_score': cup_circle.get('final_score', cup_circle['score'])
        })

# 중심축 계산 (final_score 기준 상위 5개 중앙값)
if len(circle_data) == 0:
    print("컵 원을 찾지 못했습니다.")
else:
    circle_data.sort(key=lambda x: x['final_score'], reverse=True)
    top_5_circles = circle_data[:5]
    top_5_centers = np.array([c['center'] for c in top_5_circles])
    center_axis = np.median(top_5_centers, axis=0)
    
    # 가장 점수 높은 단면의 높이를 기준점으로 설정
    best_slice = circle_data[0]  # final_score가 가장 높은 단면
    reference_z = best_slice['z']
    
    print(f"컵 중심축 (XY 평면): ({center_axis[0]:.6f}, {center_axis[1]:.6f})")
    print(f"기준 단면 높이: {reference_z:.4f}m (final_score: {best_slice['final_score']:.3f})")
    
    # 디버깅: 기준 원 중심과 중심축의 거리 확인
    best_center = best_slice['center']
    center_offset = np.linalg.norm(best_center - center_axis)
    print(f"기준 원 중심: ({best_center[0]:.6f}, {best_center[1]:.6f})")
    print(f"기준 원 중심과 중심축의 거리: {center_offset:.6f}m")
    print(f"기준 원 반지름: {best_slice['radius']:.6f}m")
    
    # 2단계: 중심축을 중심으로 하는 원 검출 함수
    def detect_circle_at_axis(slice_2d, center_axis, n_iterations=100, threshold=0.003, 
                              min_radius=0.02, max_radius=0.1):
        """
        중심축을 중심으로 하는 원을 RANSAC으로 검출
        
        Returns:
            반지름 (float) 또는 None (원이 없으면)
        """
        if len(slice_2d) < 10:
            return None
        
        # 각 포인트에서 중심축까지의 거리 계산
        distances = np.linalg.norm(slice_2d - center_axis, axis=1)
        
        best_radius = None
        best_inlier_count = 0
        best_points_per_meter = 0.0
        
        for _ in range(n_iterations):
            # 랜덤하게 1개 포인트 선택
            sample_idx = np.random.randint(len(slice_2d))
            radius_candidate = distances[sample_idx]
            
            # 반지름 범위 체크
            if not (min_radius <= radius_candidate <= max_radius):
                continue
            
            # 다른 포인트들이 그 반지름에 맞는지 확인
            inlier_mask = np.abs(distances - radius_candidate) < threshold
            inlier_count = np.sum(inlier_mask)
            
            # 둘레 길이당 포인트 수 계산
            circumference = 2 * np.pi * radius_candidate
            points_per_meter = inlier_count / circumference if circumference > 0 else 0
            
            # 가장 높은 points_per_meter를 가진 반지름 선택
            if points_per_meter > best_points_per_meter:
                best_points_per_meter = points_per_meter
                best_inlier_count = inlier_count
                best_radius = radius_candidate
        
        # 원 판정 (둘레 길이 기준)
        if best_radius is not None:
            # 최소 포인트 밀도 체크 (10개/m)
            min_points_per_meter = 10.0
            if best_points_per_meter >= min_points_per_meter and best_inlier_count >= 10:
                return best_radius
            else:
                return None
        else:
            return None
    
    # 3단계: 기준 높이에서 먼저 원 검출 시도
    all_heights = points[:, 2]
    z_min = all_heights.min()
    z_max = all_heights.max()
    
    print(f"\n전체 높이 범위: {z_min:.4f}m ~ {z_max:.4f}m")
    
    # 기준 높이에서 먼저 원 검출 시도
    print(f"\n=== 기준 높이({reference_z:.4f}m)에서 원 검출 시도 ===")
    reference_slice_mask = (points[:, 2] >= reference_z - thickness) & (points[:, 2] <= reference_z + thickness)
    reference_slice_points = points[reference_slice_mask]
    print(f"기준 높이 단면 포인트 개수: {len(reference_slice_points)}개")
    
    volume_data = []
    consecutive_failures = 0
    max_consecutive_failures = 3  # 연속으로 3개 실패하면 컵이 끝난 것으로 판정
    cup_started = False  # 컵이 시작되었는지 여부
    
    if len(reference_slice_points) >= 10:
        reference_slice_2d = reference_slice_points[:, :2]
        reference_radius = detect_circle_at_axis(reference_slice_2d, center_axis, 
                                                 n_iterations=100, 
                                                 threshold=0.003)
        
        if reference_radius is not None:
            print(f"✓ 기준 높이에서 원 검출 성공! 반지름: {reference_radius:.6f}m")
            # 기준 높이에서 원이 검출됨 → volume_data에 추가하고 cup_started = True
            cup_started = True
            area = np.pi * reference_radius ** 2
            volume_data.append({
                'z': reference_z,
                'radius': reference_radius,
                'area': area
            })
            
            # 기준 높이를 중심으로 위아래로 별도 탐색 (기준 높이는 제외)
            # 위쪽: reference_z → z_max (오름차순, reference_z 제외)
            z_above = np.linspace(reference_z, z_max, 50)[1:]  # reference_z 제외
            # 아래쪽: reference_z → z_min (내림차순, reference_z 제외)
            z_below = np.linspace(reference_z, z_min, 50)[1:]  # reference_z 제외, 내림차순
        else:
            print(f"✗ 기준 높이에서 원 검출 실패")
            # 중심축에서 각 포인트까지의 거리 분포 확인
            distances = np.linalg.norm(reference_slice_2d - center_axis, axis=1)
            print(f"  거리 범위: {distances.min():.6f}m ~ {distances.max():.6f}m")
            print(f"  평균 거리: {distances.mean():.6f}m")
            print(f"  표준편차: {distances.std():.6f}m")
            
            # 기준 높이에서 원이 검출 안 됨 → 기준 높이를 포함하여 탐색
            z_below = np.linspace(z_min, reference_z, 50)
            z_above = np.linspace(reference_z, z_max, 50)
            z_range_detailed = np.concatenate([z_below, z_above[1:]])  # reference_z는 한 번만
    else:
        print(f"✗ 기준 높이 단면 포인트 부족 ({len(reference_slice_points)}개)")
        # 기준 높이를 포함하여 탐색
        z_below = np.linspace(z_min, reference_z, 50)
        z_above = np.linspace(reference_z, z_max, 50)
        z_range_detailed = np.concatenate([z_below, z_above[1:]])  # reference_z는 한 번만
    
    # 탐색 함수 정의
    def search_direction(z_range, direction_name):
        """한 방향으로 탐색하는 함수"""
        local_consecutive_failures = 0
        found_count = 0
        
        for z in z_range:
            h_min = z - thickness
            h_max = z + thickness
            
            height_mask = (points[:, 2] >= h_min) & (points[:, 2] <= h_max)
            slice_points = points[height_mask]
            
            if len(slice_points) < 10:
                local_consecutive_failures += 1
                if local_consecutive_failures >= max_consecutive_failures:
                    print(f"  {direction_name} 방향: 연속 {max_consecutive_failures}개 실패로 중단 (마지막 높이: {z:.4f}m)")
                    break
                continue
            
            slice_2d = slice_points[:, :2]
            radius = detect_circle_at_axis(slice_2d, center_axis, 
                                           n_iterations=100, 
                                           threshold=0.003)
            
            if radius is not None:
                local_consecutive_failures = 0
                found_count += 1
                area = np.pi * radius ** 2
                volume_data.append({
                    'z': z,
                    'radius': radius,
                    'area': area
                })
            else:
                local_consecutive_failures += 1
                if local_consecutive_failures >= max_consecutive_failures:
                    print(f"  {direction_name} 방향: 연속 {max_consecutive_failures}개 실패로 중단 (마지막 높이: {z:.4f}m)")
                    break
        
        return found_count
    
    # 탐색 실행
    if cup_started:
        print(f"기준 높이에서 원 검출됨 → 위아래로 별도 탐색")
        print(f"\n=== 위쪽 탐색 시작 (기준 높이: {reference_z:.4f}m → 최대 높이: {z_max:.4f}m) ===")
        above_count = search_direction(z_above, "위쪽")
        print(f"위쪽 탐색 완료: {above_count}개 단면 검출")
        
        print(f"\n=== 아래쪽 탐색 시작 (기준 높이: {reference_z:.4f}m → 최소 높이: {z_min:.4f}m) ===")
        below_count = search_direction(z_below, "아래쪽")
        print(f"아래쪽 탐색 완료: {below_count}개 단면 검출")
    else:
        print(f"기준 높이에서 원 검출 안 됨 → 전체 탐색")
        print(f"탐색 범위: {z_range_detailed[0]:.4f}m ~ {z_range_detailed[-1]:.4f}m (총 {len(z_range_detailed)}개)")
        print(f"\n=== 전체 단면 탐색 시작 ===")
        
        consecutive_failures = 0
        for z in z_range_detailed:
            h_min = z - thickness
            h_max = z + thickness
            
            height_mask = (points[:, 2] >= h_min) & (points[:, 2] <= h_max)
            slice_points = points[height_mask]
            
            if len(slice_points) < 10:
                consecutive_failures += 1
                if cup_started and consecutive_failures >= max_consecutive_failures:
                    break
                continue
            
            slice_2d = slice_points[:, :2]
            radius = detect_circle_at_axis(slice_2d, center_axis, 
                                           n_iterations=100, 
                                           threshold=0.003)
            
            if radius is not None:
                if not cup_started:
                    print(f"  첫 원 검출: 높이 {z:.4f}m, 반지름 {radius:.6f}m")
                    cup_started = True
                consecutive_failures = 0
                area = np.pi * radius ** 2
                volume_data.append({
                    'z': z,
                    'radius': radius,
                    'area': area
                })
            else:
                if cup_started:
                    consecutive_failures += 1
                    if consecutive_failures >= max_consecutive_failures:
                        print(f"  연속 {max_consecutive_failures}개 실패로 탐색 중단 (마지막 높이: {z:.4f}m)")
                        break
    
    # 4단계: 부피 계산 (단면적분)
    if len(volume_data) > 0:
        volume_data.sort(key=lambda x: x['z'])
        
        total_volume = 0.0
        for i in range(len(volume_data) - 1):
            z1 = volume_data[i]['z']
            z2 = volume_data[i + 1]['z']
            area1 = volume_data[i]['area']
            area2 = volume_data[i + 1]['area']
            
            h = z2 - z1
            # 사다리꼴 공식: V = (A1 + A2) / 2 * h
            segment_volume = (area1 + area2) / 2 * h
            total_volume += segment_volume
        
        # 결과 출력
        print(f"검출된 컵 단면 개수: {len(volume_data)}개")
        print(f"높이 범위: {volume_data[0]['z']:.4f}m ~ {volume_data[-1]['z']:.4f}m")
        print(f"총 높이: {volume_data[-1]['z'] - volume_data[0]['z']:.4f}m")
        print(f"평균 반지름: {np.mean([v['radius'] for v in volume_data]):.6f}m")
        print(f"반지름 범위: {min([v['radius'] for v in volume_data]):.6f}m ~ {max([v['radius'] for v in volume_data]):.6f}m")
        
        # 각 단면의 반지름 디버깅 출력
        print(f"\n=== 각 단면의 반지름 ===")
        for i, v in enumerate(volume_data):
            print(f"단면 {i+1:3d}: 높이 {v['z']:8.4f}m, 반지름 {v['radius']:8.6f}m, 면적 {v['area']:10.6f}m²")
        
        print(f"\n계산된 부피: {total_volume * 1000:.2f} mL")
        print(f"계산된 부피: {total_volume * 1e6:.2f} cm³")
    else:
        print("중심축을 중심으로 하는 원을 찾지 못했습니다.")